In [13]:
from splink import Linker, DuckDBAPI
import pandas as pd

db_api = DuckDBAPI()

In [14]:
locs_df = pd.read_csv("production_locations_ds.csv")
locs_df = locs_df.drop(columns=[
    "created_at",
    "updated_at",
    "potential_match_score",
    "potential_match_os_id",
    "found_index"
])

sample_df = locs_df.sample(30000)

In [15]:
import json

with open("record_linkage_model_beta.json", "r") as file:
    record_linkage_model = json.load(file)

linker = Linker(
    sample_df,
    settings=record_linkage_model,
    db_api=db_api,
)

In [16]:
df_predictions = linker.inference.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe()

Blocking time: 12.39 seconds
Predict time: 47.55 seconds


,match_weight,match_probability,os_id_l,os_id_r,name_l,name_r,gamma_name,tf_name_l,tf_name_r,bf_name,...,geocoded_location_type_l,geocoded_location_type_r,lng_l,lng_r,lat_l,lat_r,gamma_lat_lng,bf_lat_lng,country_code_l,country_code_r
0,-1.982279,0.201973,TN20250733AB2K2,TN2025073ENAZYZ,STE SARRATEX,STE ECOPLAST TUNISIE,1,0.000033,0.000033,1.683499,...,APPROXIMATE,APPROXIMATE,10.571809,10.571809,35.874674,35.874674,5,21.796929,TN,TN
1,-1.279323,0.291779,CA20242633EHQCW,CA2024263M96R5Q,Ferme Écologique Guimond Inc. (#738–#739),La Ferme l'Entrepont (Sennc) (#916),1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,-71.139205,-71.139205,46.496540,46.496540,5,21.796929,CA,CA
2,-1.982279,0.201973,KZ2025053T6CNXM,KZ2025053W76GAS,MEBEL LORD LLP,MZK_2023 LLP,1,0.000033,0.000033,1.683499,...,APPROXIMATE,APPROXIMATE,66.923684,66.923684,48.019573,48.019573,5,21.796929,KZ,KZ
3,-1.860335,0.215940,TR20223537M26TM,TR2022353RFN9KF,GÜLER FLEXİBLE AMBALAJ SAN. VE TİC. A.Ş.,TALİA EXPORT MAKİNA İÇ VE DIŞ TİC. LTD. ŞTİ.,1,0.000033,0.000033,1.683499,...,ROOFTOP,RANGE_INTERPOLATED,37.428947,37.431825,37.094845,37.088824,2,23.719415,TR,TR
4,-1.860335,0.215940,TR202235355BAYM,TR2022353RFN9KF,HASPAK KAĞITÇILIK SAN. VE TİC. LTD.ŞTİ.,TALİA EXPORT MAKİNA İÇ VE DIŞ TİC. LTD. ŞTİ.,1,0.000033,0.000033,1.683499,...,RANGE_INTERPOLATED,RANGE_INTERPOLATED,37.441028,37.431825,37.086841,37.088824,2,23.719415,TR,TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,-1.860335,0.215940,US2024312F83P7D,US20243173V1W7X,LAKEWOOD SEIFER LLC,AMERICAN BUSINESS PAPER INC.,1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,-74.212178,-74.216240,40.081477,40.082420,2,23.719415,US,US
4225,-0.028231,0.495108,US2024246VAQE9F,US2024246ZRK5AN,CON EDISON - SERVICE BOX 58219,CON EDISON - SERVICE BOX 48697,5,0.000033,0.000033,4346.434584,...,None,None,-73.937822,-73.945710,40.852134,40.777860,-1,1.000000,US,US
4226,-1.982279,0.201973,US2024247B3BYBZ,US2024284R26TH9,"S E M & Bros, Inc.","Parade Produce, Inc.",1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,-76.784730,-76.784730,39.165855,39.165855,5,21.796929,US,US
4227,-1.860335,0.215940,US202431812B71J,US20243183CGZ3X,UNITED A&S JEWELRY INC.,L'ART DE JEWEL LTD,1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,-73.980367,-73.981460,40.757645,40.762572,2,23.719415,US,US


In [17]:
lk_df = df_predictions.as_pandas_dataframe()

lk_df[lk_df["match_probability"] > 0.8]

,match_weight,match_probability,os_id_l,os_id_r,name_l,name_r,gamma_name,tf_name_l,tf_name_r,bf_name,...,geocoded_location_type_l,geocoded_location_type_r,lng_l,lng_r,lat_l,lat_r,gamma_lat_lng,bf_lat_lng,country_code_l,country_code_r
100,2.219989,0.823287,KZ20250564E68A3,KZ2025056DF1S3J,AQTAQYR –BEREKE AGRICULTURE PRODUCTION COOPERA...,QANYŞ-1 AGRICULTURAL PRODUCTION COOPERATIVE,2,0.000033,0.000033,19.037542,...,APPROXIMATE,APPROXIMATE,80.139009,80.139009,48.943397,48.943397,5,21.796929,KZ,KZ
109,3.004597,0.889203,TR20223531PY5Q8,TR20230121X0GA8,POLEN MAKİNA END.AMB.SAN.VE DIŞ TİC.LTD.ŞTİ.,NELSON İÇ VE DIŞ TİC.LTD.ŞTİ.,2,0.000033,0.000033,19.037542,...,ROOFTOP,ROOFTOP,37.428355,37.439005,37.094313,37.087832,1,6.185203,TR,TR
128,3.002357,0.889050,US202431701WS3M,US2024317DQRQHZ,METRO DYEING LLC,EUROCO COSTUMES INC,1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,-73.992441,-73.992441,40.754804,40.754804,5,21.796929,US,US
148,4.905822,0.967719,US2024246FRNRYE,US2024246VAQE9F,CON EDISON - SERVICE BOX 52132,CON EDISON - SERVICE BOX 58219,5,0.000033,0.000033,4346.434584,...,None,None,-73.981062,-73.937822,40.788513,40.852134,-1,1.000000,US,US
167,3.124300,0.897118,TR20223522JSVZX,TR20230121X0GA8,CANAN TEKSTİL SAN. VE TİC. A.Ş.,NELSON İÇ VE DIŞ TİC.LTD.ŞTİ.,1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,37.441656,37.439005,37.082448,37.087832,2,23.719415,TR,TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4101,4.905822,0.967719,US2024246YSWRX4,US2024247XS6HQ1,CON EDISON - SERVICE BOX 4219,CON EDISON - SERVICE BOX 10908,5,0.000033,0.000033,4346.434584,...,None,None,-73.999356,-73.970721,40.745644,40.760712,-1,1.000000,US,US
4138,4.905822,0.967719,US202424616V659,US2024246VAQE9F,CON EDISON - SERVICE BOX 51474,CON EDISON - SERVICE BOX 58219,5,0.000033,0.000033,4346.434584,...,None,None,-73.857310,-73.937822,40.751175,40.852134,-1,1.000000,US,US
4147,2.219989,0.823287,IN2025028MDXREE,IN2025029A9R280,MANICKAM BLUE METAL MINES & MINERALS CRUSHER I...,SVPCR BLUE METAL MINES & MINERALS CRUSHER IMPE...,2,0.000033,0.000033,19.037542,...,ROOFTOP,ROOFTOP,80.216468,80.216468,13.127815,13.127815,5,21.796929,IN,IN
4152,3.124300,0.897118,TR20223539NF3S0,TR20230121X0GA8,AVANTAJ HIZLI TÜKETİM ÜRÜNLERİ SATIŞ PAZ. VE T...,NELSON İÇ VE DIŞ TİC.LTD.ŞTİ.,1,0.000033,0.000033,1.683499,...,ROOFTOP,ROOFTOP,37.441656,37.439005,37.082448,37.087832,2,23.719415,TR,TR


In [18]:
lk_df[lk_df["match_probability"] > 0.8].to_csv("lk_df.csv", index=False)